kmeans of LSeg conceptfusion random_sam random_grounded_sam

In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from PIL import Image

from core.integrate import PanopticFusionScalableTSDFVolume, FeatureFusionScalableTSDFVolume

In [3]:
draw = o3d.visualization.EV.draw

o3d._build_config["ENABLE_HEADLESS_RENDERING"]

True

In [4]:
device = "cuda:1"
tsdf_volume = PanopticFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=device,
)

scene = "scannet_scene0000_00"
load_dir = "/home/quanta/Experiments/feature-instance-fusion/" + scene + "/"
tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")

In [5]:
verts, faces = tsdf_volume.extract_mesh()
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)

In [6]:
tsdf_volume.reset_instance()
tsdf_volume.load_instance(load_dir + 'tsdf_panoptic_gt_instance/panoptic_labels.pt')

In [7]:
json_pth = os.path.abspath("../../config/views/" + scene + ".json")
with open(json_pth, "r") as f:
    views = json.load(f)['views']

### KMeans of SAM

In [ ]:
kmeans_sam = torch.load(load_dir + "kmeans_random_sam/kmeans_labels.pt").to(device)
tsdf_volume._instance = kmeans_sam
labels = tsdf_volume.extract_label_on_grid(verts=verts)[0]
color_set = np.random.random(size=(labels.max() + 1, 3))
color = color_set[labels]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [ ]:
save_dir = "../04_kmeans/01_random_sam/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

KMeans of grounded sam

In [ ]:
kmeans = torch.load(load_dir + "kmeans_random_grounded_sam/kmeans_labels.pt").to(device)
tsdf_volume._instance = kmeans
labels = tsdf_volume.extract_label_on_grid(verts=verts)[0]
color_set = np.random.random(size=(labels.max() + 1, 3))
color = color_set[labels]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [ ]:
save_dir = "../04_kmeans/02_random_grounded_sam/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

### Kmeans of LSeg

In [ ]:
kmeans = torch.load(load_dir + "kmeans_lseg_pruned/kmeans_labels.pt").to(device)
tsdf_volume._instance = kmeans
labels = tsdf_volume.extract_label_on_grid(verts=verts)[0]
color_set = np.random.random(size=(labels.max() + 1, 3))
color = color_set[labels]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [ ]:
save_dir = "../04_kmeans/03_lseg/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

### Kmeans of concepftfusion

In [11]:
kmeans = torch.load(load_dir + "kmeans_conceptfusion/kmeans_labels.pt").to(device)
tsdf_volume._instance = kmeans
labels = tsdf_volume.extract_label_on_grid(verts=verts)[0]
color_set = np.random.random(size=(labels.max() + 1, 3))
color = color_set[labels]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [9]:
save_dir = "../04_kmeans/04_concepftfusion/"
os.makedirs(save_dir, exist_ok=True)

In [10]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")